In [1]:
#Onnx Çalışma Zamanı, bir transformatör modeli yüklenirken çoğu optimizasyonu otomatik olarak uygular.
!pip install onnxruntime transformers gdown 

     |████████████████████████████████| 4.1MB 13.4MB/s 
     |████████████████████████████████| 2.1MB 54.5MB/s 
     |████████████████████████████████| 870kB 42.1MB/s 
     |████████████████████████████████| 3.3MB 16.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=4ed0196cd260c8a3286c94b74fedfcd3ad0dbd0f6b90f9deaa561cccc94921d6
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [2]:
import onnxruntime as ort
from transformers import BertTokenizerFast
import pandas as pd
import re
import math
import sys
import os
import time
import numpy as np
import torch

In [3]:
# Oluşturulan Türkçe Duygu analizi modeli yüklenir.

!gdown --id 1NWlShIdsEb543J6yIPCyKsANAHCwSR8X

Downloading...
From: https://drive.google.com/uc?id=1NWlShIdsEb543J6yIPCyKsANAHCwSR8X
To: /content/distilbert.onnx
270MB [00:01, 197MB/s]


In [4]:
# Türkçe Duygu Analizi için oluşturulan Model ve Tokenizer yüklenir. 

ort_session = ort.InferenceSession("distilbert.onnx")
tokenizer = BertTokenizerFast.from_pretrained('dbmdz/distilbert-base-turkish-cased')

In [5]:
# Veri seti yüklenir.
dataset = pd.read_csv('ses_kayitlari_metin.csv' , sep='delimiter',  engine='python', encoding='cp1254')

data = dataset['sentence'].values.tolist()

In [7]:
#Öncelikle veriler üzerinde tokenleştirme işlemleri gerçekleştirilir.
#Modele belirlenen boyutlarda veri gönderimi sağlanır.
#Maskeleme işlemleri gerçekleştirilir ve belirlenen veri gönderim boyutu ile veri seti üzerinde işlemler yapılır.
#Veri seti üzerinde yapılan işlemler sonrasında final_results değişkenine verilerin duygu etiketleri aktarılır.


tokenized_sentences = tokenizer.batch_encode_plus(data, padding="max_length", truncation=True, max_length=128)

batch_size = 40
N = len(data)
final_results = []
for idx in range(math.ceil(N/batch_size)):
    print("Inference:",len(final_results),"/",N)
    inp_ids = tokenized_sentences["input_ids"][idx*batch_size:min((idx+1)*batch_size, N)]
    att_mask = tokenized_sentences["attention_mask"][idx*batch_size:min((idx+1)*batch_size, N)]
    real_size = len(inp_ids)
    if len(inp_ids)<batch_size:
        inp_ids.extend([inp_ids[0]]*(batch_size-real_size))
        att_mask.extend([att_mask[0]]*(batch_size-real_size))
    x = {'input_ids': np.array(inp_ids), 'input_atns':np.array(att_mask)}
    outputs = ort_session.run(None, x)
    values = np.argmax((outputs[0]), 1)[:real_size]
    final_results.extend([x-1 for x in values.tolist()])

Inference: 0 / 5001
Inference: 40 / 5001
Inference: 80 / 5001
Inference: 120 / 5001
Inference: 160 / 5001
Inference: 200 / 5001
Inference: 240 / 5001
Inference: 280 / 5001
Inference: 320 / 5001
Inference: 360 / 5001
Inference: 400 / 5001
Inference: 440 / 5001
Inference: 480 / 5001
Inference: 520 / 5001
Inference: 560 / 5001
Inference: 600 / 5001
Inference: 640 / 5001
Inference: 680 / 5001
Inference: 720 / 5001
Inference: 760 / 5001
Inference: 800 / 5001
Inference: 840 / 5001
Inference: 880 / 5001
Inference: 920 / 5001
Inference: 960 / 5001
Inference: 1000 / 5001
Inference: 1040 / 5001
Inference: 1080 / 5001
Inference: 1120 / 5001
Inference: 1160 / 5001
Inference: 1200 / 5001
Inference: 1240 / 5001
Inference: 1280 / 5001
Inference: 1320 / 5001
Inference: 1360 / 5001
Inference: 1400 / 5001
Inference: 1440 / 5001
Inference: 1480 / 5001
Inference: 1520 / 5001
Inference: 1560 / 5001
Inference: 1600 / 5001
Inference: 1640 / 5001
Inference: 1680 / 5001
Inference: 1720 / 5001
Inference: 1760 /

In [8]:
#Modelde üç duygu etiketi bulunmaktadır.Bunlar ; Negatif (-1), Nötr (0) ve Pozitiftir(1).
#Veri setinde bulunan cümleler üzerinde yapılan tokenleştirme, maskeleme ve modelde yapılan işlemler sonucunda duygu tespitleri sağlanır.
#Sentence ve Sentiment şeklinde iki sütuna ayrılan çıktılar, excel dosyasına aktarılır.

df = pd.DataFrame(columns=["Sentence", "Sentiment"])

df["Sentence"] = data
df["Sentiment"] = final_results
print(df)
df.to_excel('BERT.xlsx')

                                               Sentence  Sentiment
0     Hayatta küçük şeyleri kovalıyor ve yine küçük ...         -1
1                 Seydiu şimdi iki mevkiyi de kaybetti.         -1
2     Fakat öte yandan, hayatta gelişim sağlayabilme...          0
3                     Seferler Haziran ayında başlıyor.          0
4            Siyasette temiz insanlara ihtiyacımız var.          1
...                                                 ...        ...
4996                    Kamuoyu ikiye bölünmüş durumda.         -1
4997          Ancak Hiseni suçlamaktan da geri kalmadı.         -1
4998           Kosova başkentindeki yolcu sayısı arttı.          0
4999                Bu tamamen Kosova yönetimine bağlı.         -1
5000        Bu olay daha büyük yankılara yol açacak mı?         -1

[5001 rows x 2 columns]


In [9]:
df['Sentiment'].value_counts() #2430 nötr, 1989 negatif ve 582 pozitif cümle elde edilmiştir.

 0    2430
-1    1989
 1     582
Name: Sentiment, dtype: int64